In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import glob
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
# set to *.csv to process all
path_to_csv = '../../data/BTW17_Twitter/hashtags/*.csv'
file_list = glob.glob(path_to_csv)

df = pd.DataFrame()

# concatenate dataframes
for index in tqdm(range(len(file_list))):
    file = file_list[index]
    
    # read file to df if first file
    if index==0:
        df = pd.read_csv(file)
        df.drop('Unnamed: 0', axis=1, inplace=True)
    
    # append to df if not first file
    else:
        df2 = pd.read_csv(file)
        df2.drop('Unnamed: 0', axis=1, inplace=True)
        df = df.append(df2)

df = df.groupby(['date', 'hashtag'], as_index=False).sum('count')

  0%|          | 0/1308 [00:00<?, ?it/s]

In [3]:
# get peak indices
peak_df = pd.read_csv('../../data/BTW17_Twitter/peaks/peaks.csv')
peak_df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [4]:
# mark peaks in dataframe and save to complete_df
hashtag_list = df['hashtag'].unique().tolist()

complete_df = pd.DataFrame()

for i in tqdm(range(len(hashtag_list))):
    hashtag = hashtag_list[i]
    small_df = df[df['hashtag']==hashtag].reset_index()
    small_peak_df = peak_df[peak_df['hashtag']==hashtag].reset_index()
    small_df['peak'] = 0
    for peak in small_peak_df['peak'].tolist():
        small_df.loc[int(peak), 'peak'] = 1
    complete_df = complete_df.append(small_df)
    

  0%|          | 0/30595 [00:00<?, ?it/s]

In [5]:
complete_df.head()

,index,date,hashtag,count,peak
0,0,2017-05-29,150jahrekapital,1,0
0,1,2017-05-29,a19,1,0
0,2,2017-05-29,abschiebung,14,0
1,1725,2017-05-31,abschiebung,19,0
2,2613,2017-06-01,abschiebung,23,0


In [6]:
# get hashtag and peak +- 3 days for topic modeling
hashtag_list = df['hashtag'].unique().tolist()
lda_dates = []    

for i in tqdm(range(len(hashtag_list))):
    hashtag = hashtag_list[i]
    peak_dates = complete_df[(complete_df['peak']==1)&(complete_df['hashtag']==hashtag)]['date'].tolist()
    for peak in peak_dates:
        start = str(datetime.strptime(peak, '%Y-%m-%d').date() - timedelta(days=3))
        end = str(datetime.strptime(peak, '%Y-%m-%d').date() + timedelta(days=3))
        daterange = pd.date_range(start, end)
        output_dates = []
        for date in daterange:
            output_dates.append(date.strftime('%Y-%m-%d'))
    if peak_dates:
        lda_dates.append(output_dates)
    else:
        lda_dates.append(peak_dates)

output_df = pd.DataFrame({'hashtag': hashtag_list, 'lda_dates': lda_dates})

  0%|          | 0/30595 [00:00<?, ?it/s]

In [7]:
# save to csv
path_file = '../../data/BTW17_Twitter/peaks/hashtags_dates.csv'
output_df.to_csv(path_file)